### Initiation

In [ ]:
!wget https://download.oracle.com/otn_software/linux/instantclient/2114000/instantclient-basic-linux.x64-21.14.0.0.0dbru.zip
!wget https://download.oracle.com/otn_software/linux/instantclient/2114000/instantclient-sdk-linux.x64-21.14.0.0.0dbru.zip

!unzip instantclient-basic-linux.x64-21.14.0.0.0dbru.zip
!unzip instantclient-sdk-linux.x64-21.14.0.0.0dbru.zip

!rm instantclient-basic-linux.x64-21.14.0.0.0dbru.zip
!rm instantclient-sdk-linux.x64-21.14.0.0.0dbru.zip

!sudo apt-get update
!sudo apt-get install libaio1

!pip install cx_Oracle
!pip install python-dotenv
!pip install sqlalchemy



In [6]:
#!/bin/bash

# Use sudo to update the Oracle Instant Client library path
!echo "/workspaces/Database-AI-Integration/instantclient_21_14" | sudo tee /etc/ld.so.conf.d/oracle-instantclient.conf

# Use sudo to update the shared library cache
!sudo ldconfig

# Export environment variables if necessary
!export ORACLE_HOME=/workspaces/Database-AI-Integration/instantclient_21_14
!export PATH=$ORACLE_HOME:$PATH
!export LD_LIBRARY_PATH=$ORACLE_HOME

!echo "Oracle Instant Client setup completed."

/workspaces/Database-AI-Integration/instantclient_21_14
Oracle Instant Client setup completed.


### Connection details

In [7]:
# Initialize and connection details
import cx_Oracle
from dotenv import load_dotenv
import os

# Initialize the Oracle client with the library directory
cx_Oracle.init_oracle_client(lib_dir="/workspaces/Database-AI-Integration/instantclient_21_14")

# Connection details
load_dotenv()

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
port = os.getenv('PORT')
service_name = os.getenv('SERVICE_NAME')

# Create a connection string (DSN)
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

# Establish the connection
connection = cx_Oracle.connect(username, password, dsn=dsn)
print("Connection established successfully!")



Connection established successfully!


### Fetching all Schema Names

In [ ]:
from sqlalchemy import create_engine, inspect
from prettytable import PrettyTable

# Create the connection string
connection_string = f"oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Connect to the database and fetch schema names
with engine.connect() as connection:
    inspector = inspect(connection)
    schema_names = inspector.get_schema_names()

# Create a PrettyTable object
table = PrettyTable()
table.field_names = ["Schema Names"]

# Add schema names to the table
for schema in schema_names:
    table.add_row([schema])

# Print the table
print(table)


### Fetching all table names

In [ ]:
from sqlalchemy import create_engine, inspect
from prettytable import PrettyTable

# Create the connection string
connection_string = f"oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Connect to the database and fetch table names
with engine.connect() as connection:
    inspector = inspect(connection)
    table_names = inspector.get_table_names()

# Create a PrettyTable object
table = PrettyTable()
table.field_names = ["Table Names"]

# Add table names to the table
for name in table_names:
    table.add_row([name])

# Print the table
print(table)


### Troubleshooting

In [ ]:
from sqlalchemy import create_engine, text

# Construct the connection string for Oracle
connection_string = f'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}'

# Establish connection to your database
engine = create_engine(connection_string)

# List of table names
table_names = ['AD_TDOCCOMPR']
#, 'AD_TDETCOMPR', 'RE_TDOCVENTA', 'RE_TDETVENTA']

# Function to fetch and print metadata for a table
def fetch_and_print_metadata(table_name):
    # Query to get columns
    columns_query = text(f"""
    SELECT column_name, data_type
    FROM all_tab_columns 
    WHERE table_name = '{table_name.upper()}'
    """)
    
    # Query to get column comments
    comments_query = text(f"""
    SELECT column_name, comments 
    FROM all_col_comments 
    WHERE table_name = '{table_name.upper()}'
    """)
    
    with engine.connect() as connection:
        print(f"Metadata for table: {table_name}")
        
        # Fetch and print column names and data types
        result = connection.execute(columns_query)
        print("Columns:")
        for row in result:
            print(row)
        
        # Fetch and print column comments
        result = connection.execute(comments_query)
        print("Comments:")
        for row in result:
            print(row)
        
        print("\n")

# Fetch and print metadata for each table
for table in table_names:
    fetch_and_print_metadata(table)
